<a href="https://colab.research.google.com/github/Berusov/Event_predict/blob/main/%D0%9C%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C_%D0%BF%D1%80%D0%BE%D0%B3%D0%BD%D0%BE%D0%B7%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F_%D1%83%D1%81%D0%BF%D0%B5%D1%88%D0%BD%D0%BE%D1%81%D1%82%D0%B8_%D0%BC%D0%B5%D1%80%D0%BE%D0%BF%D1%80%D0%B8%D1%8F%D1%82%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold

# Загрузка данных
url = "https://raw.githubusercontent.com/Berusov/Event_predict/main/events_hall_dataset_with_scores.csv"
data = pd.read_csv(url)


# Признаки и целевая переменная
features = ["duration", "audience_size", "award_duration", "num_items", "perf_duration", "host_speech_duration"]
target = "success_score"

X = data[features]
y = data[target]

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Конвейер с масштабированием и моделью
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Стандартизация данных
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Добавление полиномиальных признаков
    ('model', GradientBoostingRegressor(random_state=42))  # Применение Gradient Boosting
])

# Кросс-валидация с 5 фолдами для оценки модели
cv = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)
print(f"Средняя абсолютная ошибка (MAE) на кросс-валидации: {-cv_scores.mean():.2f}")

# Оптимизация гиперпараметров с помощью GridSearchCV
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'model__max_depth': [3, 5, 7],
    'model__min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Лучшая модель
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Оценка модели
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Средняя абсолютная ошибка модели: {mae:.2f}")
print(f"Коэффициент детерминации (R²): {r2:.2f}")

# Функция для предсказания успешности нового мероприятия
def predict_success(event_features):
    event_df = pd.DataFrame([event_features], columns=features)
    predicted_score = best_model.predict(event_df)[0]
    return predicted_score


Средняя абсолютная ошибка (MAE) на кросс-валидации: 0.35
Лучшие параметры: {'model__learning_rate': 0.05, 'model__max_depth': 5, 'model__min_samples_split': 10, 'model__n_estimators': 300}
Средняя абсолютная ошибка модели: 0.20
Коэффициент детерминации (R²): 0.97


In [4]:
# Пример предсказания
test_event = {
    "duration": 70,  # Продолжительность мероприятия (мин)
    "audience_size": 1331,  # Размер аудитории (чел)
    "award_duration": 20,  # Длительность награждения (мин)
    "num_items": 8,  # Количество номеров/активностей
    "perf_duration": 40,  # Длительность выступлений (мин)
    "host_speech_duration": 10  # Длительность речи ведущих (мин)
}

test_score = predict_success(test_event)
print(f"Прогнозируемая успешность мероприятия: {test_score:.2f}")

Прогнозируемая успешность мероприятия: 9.28
